# Homework 5: Oliver Muellerklein

Collaboration with Jacob Bukoski and Zhongqi Miao.

In [4]:
import sys

#sys.path.append('/usr/local/lib/python3.5/site-packages')
#sys.path

## SQL x Weather Underground

The first steps are to import some packages for later and create a helper / utility class for SQLite.


In [112]:
import pandas as pd
import numpy as np
import sqlite3
from bs4 import BeautifulSoup
from urllib.request import urlopen
from datetime import datetime
import matplotlib.pyplot as plt
#%matplotlib inline

"""Helper utility class for SQL:

    * connect(..., database)
    
    * table_list(...)
    
    * drop(..., table)
    
    * close(...)
"""
class UtilSQL:
    
    def __init__(self):
        self.db = None
        self.connection = None
        self.cursor = None
        
    def connect(self, database):
        self.db = database
        self.connection = sqlite3.connect(self.db)
        self.cursor = self.connection.cursor()
        
    def tableList(self):
        sqlCMD = "select * from sqlite_master where type = 'table';"
        self.cursor.execute(sqlCMD)
        dbMeta = self.cursor.fetchall()
        for entry in dbMeta: 
            print(entry)
            
    def drop(self, table):
        sqlCMD = "drop table %s" % (table)
        self.cursor.execute(sqlCMD)
            
    def close(self):
        self.connection.commit()
        self.connection.close()



### Extract clean dataframe from CSV

Next we want to use the given CSVs and extract a cleaner table / data structure for later use.


In [117]:
# Read in data from CSV
dataHead = pd.read_csv('hw_5_data/top_airports.csv')
icao = pd.read_csv('hw_5_data/ICAO_airports.csv')
icao = icao.rename(columns = {'iata_code' : 'IATA'})
dataMerge = pd.merge(dataHead, icao, on = ['IATA'])
dataFin = dataMerge[['ICAO', 'Airport', 'City', 'latitude_deg', 'longitude_deg', 'elevation_ft']]

### Initialize connection to SQL and begin!

Now we can begin to fill in our tables with a connection to our SQL database through our utility class. More below...

In [118]:
# Connect to SQL table
sql = UtilSQL()
sql.connect('HW5.db')

# Run SQL commands 

sqlCMD = """CREATE TABLE Top_Airport (ICAO TEXT, Airport TEXT,
            City TEXT, Latitude REAL, Longitude REAL, Elevation REAL,
            PRIMARY KEY (ICAO))"""
#sql.cursor.execute(sqlCMD)

#for i in range(len(dataFin)):
#    sqlCMD = ("INSERT INTO Top_Airport (ICAO, Airport, City, Latitude, Longitude, Elevation) VALUES " 
#               + str(tuple(dataFin.ix[i])))
#    sql.cursor.execute(sqlCMD)

sqlCMD = 'select * from Top_Airport'

sql.cursor.execute(sqlCMD)

# Show data table (SQL table)
#sql.cursor.fetchall()

### Exploratory steps with BeautifulSoup

In [119]:
try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen
    
#response = urlopen("http://words.bighugelabs.com/")
#html = response.read()
#response.close()
# pip install beautifulsoup4
from bs4 import BeautifulSoup
#soup = BeautifulSoup(html, "html5lib")
#forms = soup.findAll("form")
#forms


We can begin with an example of how to scrape the **Weather Underground** data for a specific year (2008) using *BeautifulSoup*.


In [120]:
#response08 = urlopen("https://www.wunderground.com/history/airport/%s/%s/1/1/CustomHistory.html?dayend=31&monthend=12&yearend=%s" \
#                   % ('KATL', '2008', '2008'))
#html08 = response08.read()
#response08.close()
#soup08 = BeautifulSoup(html08, "html5lib")

# Try to pull a single year
#tbody08 = soup08.find('table', id = 'obsTable', class_ = 'responsive obs-table daily').find_all('tbody')

## A Class approach to generating weather data

There are multiple ways that the database could be connected and integrated with the web scraping of Weather Underground. The following class contains a few different methods and components that allow the data to be built either: 

**A)** outside of the class by connecting to the SQL data externally, i.e. no data is stored within an instance of this class 

or 

**B)** within the class through a mapping to the specific years, i.e. data is stored within an instance of the class 

I am providing the code here as references to all functionality but the actual data used in the remaining questions imports *hw5.db* - which is a result of using the class without any instance-specific data storage, i.e. option **A**.  

In [122]:
class Weather:
    
    def __init__(self):
        self.begin = None
        self.end = None
        self.icao = None
        self.tbody = None
        self.y = None
        self.m = None
        self.d = None
        self.icao = None
        
        self.counterr = 0
        self.monthIndx = []
        self.tryTd = None
        self.tbody08 = None
        self.tbody09 = None
        self.tbody10 = None
        self.tbody11 = None
        self.tbody12 = None
        self.tbody13 = None
        self.tbody14 = None
        self.tbody15 = None
        self.tbody16 = None
        self.yyear = None
        
        # Bunch of maps 
        self.getNumDayIncrease = {
            'Jan' : 32,
            'Feb' : 28
        }
        
        self.getTbody = {
            2008 : self.tbody08,
            2009 : self.tbody09,
            2010 : self.tbody10,
            2011 : self.tbody11,
            2012 : self.tbody12,
            2013 : self.tbody13,
            2014 : self.tbody14,
            2015 : self.tbody15,
            2016 : self.tbody16
        }

    global sql
    
    def setTbody(self):
        
        self.tbody = self.getTbody[self.yyear]
    
    def setTryTD(self):
        self.tryTd = self.tbody[self.counterr].find_all('td')[0].string
        tryIt = self.trySwitcher()

        if (tryIt):
            self.tryTd = self.tbody[self.counterr].find_all('td')[0].string
            tryIt = self.trySwitcher()
      
    
    def mapSwitcher(self, argument):
    
        switchh = {
            'Jan' : True,
            'Feb' : True
        }

        return switchh.get(argument, 'nothing')

    
    def trySwitcher(self):
        
        if (self.mapSwitcher(self.tryTd)):
            
            # Get index from tbody 
            self.monthIndx.append(self.counterr)
            self.counterr += self.getNumDayIncrease[self.tryTd]
            return False

        else:
            # Try again 
            self.counterr += 1
            return True
        
    def setYearData(self, icao, yyear):
        
        self.yyear = yyear
        self.getTbody[yyear] = self.fetch(icao, yyear)
    
    def fetch (self, icao, date):
        """
            Function from Miao: pull specific year stepwise. 
            Only connections, no internal class storage of tables.
            
            return: all tbody tags from HTML document (DOM)
        """

        response = urlopen("https://www.wunderground.com/history/airport/%s/%s/1/1/CustomHistory.html?dayend=31&monthend=12&yearend=%s" \
                           % (icao, date, date))
        html = response.read()
        response.close()
        soup = BeautifulSoup(html,"html5lib")
        tbody = soup.find('table', id = 'obsTable', class_ = 'responsive obs-table daily').find_all('tbody')
        return tbody
    
    def callback (self, row):
        """
            Function from Miao: Callback for iterative year HTML pull of data.
        """
    
        if 'avg' in row.text:
            self.m += 1
            self.d = 0

        #print(self.y, self.m, self.d)

        if self.d != 0:
            td = row.find_all('td')

            MaxT = td[2].text.strip()
            MeanT = td[1].text.strip()
            MinT = td[3].text.strip()
            MaxH = td[8].text.strip()
            MeanH = td[7].text.strip()
            MinH = td[9].text.strip()
            Prep = td[-2].text.strip()
            Date = datetime(self.y, self.m, self.d)
            
            sql.cursor.execute("""INSERT INTO Weather (ICAO, Date, MaxT, MeanT,
                            MinT, MaxH, MeanH, MinH, Prep) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)""",
                                  (self.icao, Date, MaxT, MeanT, MinT, MaxH, MeanH, MinH, Prep))

        self.d += 1

        
    
    def popData (self, begin, end, icao):
        """
            Function from Miao: populate SQL table externally.
            Steps through each year of data and increments / counts 
            months and days - accounts for leap years etc. 
        """
    
        self.icao = icao
        
        for i in range(begin, end + 1):

            tbody = self.fetch(icao, i)
            self.y = i
            self.m = 0
            self.d = 0
            a = list(map(self.callback, tbody))

Below is a beginning example of using the **Weather** class to manually step through the years and months, iterively augmenting the data to the data map. *Note: the complete code for not incorporating the data into the class (which is the method we used to directly create the database file) is included below in the comments.*

In [123]:
weatherInst = Weather()
weatherInst.setYearData('KATL', 2008)
weatherInst.setTbody()
len(weatherInst.getTbody[2008])
weatherInst.setTryTD()

# Counterr is used to count through number of days per month 
#weatherInst.counterr

### Calculate correlation coefficient directly on SQL table

The next major step is calculating the correlation coefficient across all of the data. This could be performed after extracting the data into standard *Python* data structures / code, *i.e. Numpy arrays and use of numpy.corrcoeff()*. However, for greater optimization and learning purposes, the correlation coefficients are calculated directly on the SQL table.  

In [124]:
# Figure out how to write out a correlation coefficient command here...

# sql_corrcoeff = """DO MATH corrcoeff FROM *"""

#sql.cursor.execute(sql_corrcoeff)
#sql.cursor.fetchall()